<a href="https://colab.research.google.com/github/CantoneseCounsellorChatbot/CantoneseChatbot/blob/main/chatbot_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch
!pip install simpletransformers
!pip install pytorch_pretrained_bert
# !pip install sqlite3
!git clone https://github.com/CantoneseCounsellorChatbot/CantoneseChatbot.git
# download question retrieval model
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv" -O pretrain-model.zip
!rm -rf /tmp/cookies.txt
! unzip -o -d CantoneseChatbot/ pretrain-model.zip

     |████████████████████████████████| 225kB 8.1MB/s 
     |████████████████████████████████| 1.2MB 44.2MB/s 
     |████████████████████████████████| 1.8MB 36.6MB/s 
     |████████████████████████████████| 3.3MB 49.8MB/s 
     |████████████████████████████████| 2.3MB 39.1MB/s 
     |████████████████████████████████| 8.2MB 39.8MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 122kB 58.0MB/s 
     |████████████████████████████████| 235kB 53.9MB/s 
     |████████████████████████████████| 174kB 51.4MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 133kB 48.8MB/s 
     |████████████████████████████████| 901kB 33.2MB/s 
     |████████████████████████████████| 4.2MB 41.7MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 112kB 55.4MB/s 
     |████████████████████████████████| 112kB 49.4MB/

     |████████████████████████████████| 133kB 6.9MB/s 
     |████████████████████████████████| 133kB 11.9MB/s 
     |████████████████████████████████| 7.6MB 10.6MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


Archive:  pretrain-model.zip
   creating: CantoneseChatbot/pretrain-model/
   creating: CantoneseChatbot/pretrain-model/output_BTS_re+pr05190/
   creating: CantoneseChatbot/pretrain-model/output_BTS_re+pr05190/model/
  inflating: CantoneseChatbot/pretrain-model/output_BTS_re+pr05190/model/config.json  
  inflating: CantoneseChatbot/pretrain-model/output_BTS_re+pr05190/model/BertAbsSum_14.bin  
   creating: CantoneseChatbot/pretrain-model/regression_question/
   creating: CantoneseChatbot/pretrain-model/regression_question/bestmodel/
  inflating: CantoneseChatbot/pretrain-model/regression_question/bestmodel/eval_results.txt  
  inflating: CantoneseChatbot/pretrain-model/regression_question/bestmodel/scheduler.pt  
  inflating: CantoneseChatbot/pretrain-model/regression_question/bestmodel/special_tokens_map.json  
  inflating: CantoneseChatbot/pretrain-model/regression_question/bestmodel/vocab.txt  
  inflating: CantoneseChatbot/pretrain-model/regression_question/bestmodel/tokenizer_conf

In [5]:
import sys
# from google.colab import drive
# drive.mount('/content/drive')
# sys.path.append('/content/drive/My Drive/chatbot/BertSum')
from datetime import time
import time
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from tqdm import tqdm

from transformers import BertTokenizer, BertForSequenceClassification
# import sqlite3
import re


In [6]:
def regressionReply(post,model,candidate):
  # print("regression")
  tokenizer = BertTokenizer.from_pretrained(model)
  model = BertForSequenceClassification.from_pretrained(model)
  model.to('cuda')
  data=pd.read_csv(candidate)
  advice_list = data.advice.drop_duplicates().to_list()
  text=post

  input_ids = []
  attention_masks = []
  for a in advice_list:
      encoded_dict = tokenizer.encode_plus(
                      text+"[SEP]"+a,                      # Sentence to encode.
                      add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                      max_length = 50,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',     # Return pytorch tensors.
                  )
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  batch_size = 32
  # print("chatbot:{}".format(advice))
  prediction_data = TensorDataset(input_ids, attention_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
  
  predictions  = []
  # print("regression2")
  # Predict 
  for batch in tqdm(prediction_dataloader):
      batch = tuple(t.to("cuda") for t in batch)
      b_input_ids, b_input_mask = batch
      with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy().reshape(-1).tolist()
  #     label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
      predictions+=logits
  #   true_labels.append(label_ids)

  return advice_list[np.argmax(predictions)], np.max(predictions)


In [7]:
def general(aa):
    def getQAlist():
        qaList = []
        conn = pd.read_csv("/content/CantoneseChatbot/keyword_list.csv")
        

        for index,row in conn.iterrows():
          if pd.notna(row["Q"]):
            tmp = {"Q":""+row["Q"],"A":""+row["A"]}
            qaList.append(tmp)
          else:
            genreal_reply=row["A"].split("|")
        return qaList,genreal_reply

    def answer(say,seg):
        if (say[0]=="你") and (say.find("唔")>0):
            msg = handleSpecial(say,seg)
            if msg !="":
                return msg
        return "@eliza@ " + getAnswer(say)
    def getAnswer(say):
        tmpList,general_reply=getQAlist()
        results = analyzeSay(say, tmpList, general_reply)
        
        msg=results[1]
        if msg !="":
            return msg
        else:
            return "然後呢？@发生错误@"

    def analyzeSay(say, tmpList, general_reply):
        patterns = []
        for i in range(len(tmpList)):
            qa = tmpList[i]
            qList = qa["Q"].split(" | ")
            aList = qa["A"].split("|")            
            elizakeyword = []
            for j in range(len(qList)):
                qi = qList[j]
 


                if say.find(qi) >-1:


                    elizakeyword.append(qi)
                    tt=handlePunc(say, qi)

                    tail = getTail(tt, qi)

                    replacedTail = tail.replace("我", "#")
                    replacedTail = replacedTail.replace("你", "我")
                    replacedTail = replacedTail.replace("#", "你")
                    msg = [tail, aList[np.random.randint(len(aList))].replace("*", replacedTail)+"$"+qi+"$"]
                    patterns.append(msg)
        if patterns==[]:
            patterns.append([say, general_reply[np.random.randint(len(general_reply))].replace("*", say)+"$"+"None"+"$"])


        return getRandomPattern(patterns)

#             except:
#                 print(i)


    def getRandomPattern(patterns):
        return patterns[np.random.randint(len(patterns))]
#     def getTail(say, q):
#         print("lbk")
#         r= re.compile(r"(.*){}([^?.;]*)".format(q))
#         tmp = r.match(say)
#         print(tmp)
#         if tmp:
#             return tmp[1]
#         return ""
    def getTail(say, q):
        r= r"(.*)({})([^?.;]*)".format(q)
        tmp = re.findall(r,say)
        if tmp !=[] :
            return tmp[0][2]
        return ""

    def handlePunc(say, keyword):
        punct = [",", "\\\\.", "!", "-", "\\\\?", "，", "！", "？", ":", ";", "；", "：", "。", "、", "…"]
        tmppunc="".join(punct)
        post = say.find(keyword)
        if post == -1:
            return say
        r1=r"[{}\s](.*?{}.*?)[{}\s]".format(tmppunc,keyword,tmppunc)
        r2=r"[{}\s]*(.*?{}.*?)[{}\s]".format(tmppunc,keyword,tmppunc)
        r3=r"[{}\s](.*?{}.*?)[{}\s]*".format(tmppunc,keyword,tmppunc)
        if re.findall(r1,say) !=[]:

            return re.findall(r1,say)[0]
        elif re.findall(r2,say) !=[]:

            return re.findall(r2,say)[0]
        elif re.findall(r3,say) !=[]:

            return re.findall(r3,say)[0]
        else:
            return say
    reply = answer(aa,aa)
    return reply

In [8]:
def chatbot(chatbot_params):
  params_df=pd.DataFrame(chatbot_params).T.sort_values(by=["order"])
  while True:
    text=input("input:")
    label=0
    for index, row in params_df.iterrows():
      if index == "general":
        print("chatbot: {}".format(general(text)))
        break
      elif index=="advice":
        modelpath = "/content/CantoneseChatbot/pretrain-model/regression_advice/bestmodel"
        advicepath= "/content/CantoneseChatbot/candidate/adviceall.csv"
        reply, score = regressionReply(text,modelpath,advicepath)
        if score > row["Threshold"]:
          print("chatbot: {}".format(reply))
          break
        continue
      elif index=="question":
        modelpath = "/content/CantoneseChatbot/pretrain-model/regression_question/bestmodel"
        advicepath= "/content/CantoneseChatbot/candidate/question.csv"
        reply, score = regressionReply(text,modelpath,advicepath)
        if score > row["Threshold"]:
          print("chatbot: {}".format(reply))
          break
        continue
      elif index=="restatement":
        modelpath = "/content/CantoneseChatbot/pretrain-model/regression_restatement/bestmodel"
        advicepath= "/content/CantoneseChatbot/candidate/restatement.csv"
        reply, score = regressionReply(text,modelpath,advicepath)
        if score > row["Threshold"]:
          print("chatbot: {}".format(reply))
          break
        continue
      elif index=="bertsum":
        print("chatbot: {}".format(general(text)))
        break


In [9]:
# The default response order is advice-->question-->restatement-->bertsum, you can change the order by changing the parameter b
# The threshold of "advice","question"and"restatement" ranges from 0 to 1.1
# The threshold of "bertsum" ranges from -10 to 0
chatbot_params={"advice":{"Threshold":1.0,"order":1},
                "question":{"Threshold":1.0,"order":2},
                "restatement":{"Threshold":1.0,"order":3},
                "bertsum":{"Threshold":-2,"order":4},
                "general":{"Threshold":None,"order":5}
                }

In [10]:
chatbot(chatbot_params)

input:hello


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
  0%|          | 0/143 [00:00<?, ?it/s]

regression2


100%|██████████| 143/143 [00:11<00:00, 12.05it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  1%|          | 1/139 [00:00<00:17,  7.83it/s]

regression2


100%|██████████| 139/139 [00:11<00:00, 11.94it/s]


chatbot: 你係咪有好明確嘅目標？


KeyboardInterrupt: ignored

In [ ]:
# BertSum, still have some bug
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import argparse
import logging
import os
from tqdm import tqdm, trange
import json

from preprocess import LCSTSProcessor, convert_examples_to_features, create_dataset
from model import BertAbsSum
from pytorch_pretrained_bert.tokenization import BertTokenizer
from utils import rouge
from utils import convert_to_unicode
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class InputExample(object):
    """A single training/test example."""

    def __init__(self, guid, src, tgt=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            src: string. The untokenized text of the target sequence.
            tgt: (Optional) string. The untokenized text of the target.
        """
        self.guid = guid
        self.src = src
        self.tgt = tgt

BATCH_SIZE = 1

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument("--model_path",
                    default=None,
                    type=str,
                    required=True,
                    help="The path to trained model.")
parser.add_argument("--config_path",
                    default=None,
                    type=str,
                    required=True,
                    help="The path to config file.")                    
# parser.add_argument("--eval_path",
#                     default=None,
#                     type=str,
#                     required=True,
#                     help="The path to the evaluation data. Should end with .tsv.")
parser.add_argument("--bert_model", 
                    default=None, 
                    type=str, 
                    required=True,
                    help="Bert pre-trained model selected in the list: bert-base-uncased, "
                    "bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, "
                    "bert-base-multilingual-cased, bert-base-chinese.")
# parser.add_argument("--result_path",
#                     default=None,
#                     type=str,
#                     required=True,
#                     help="The path where you save your results.")                    
parser.add_argument("--max_src_len",
                    default=130,
                    type=int,
                    help="Max sequence length for source text. Sequences will be truncated or padded to this length")
parser.add_argument("--max_tgt_len",
                    default=30,
                    type=int,
                    help="Max sequence length for target text. Sequences will be truncated or padded to this length")


if __name__ == "__main__":
    # args = parser.parse_args()
    args = parser.parse_args(args=['--model_path','/content/drive/MyDrive/chatbot/pretrain-model/output_BTS_re+pr05190/model/BertAbsSum_14.bin',
                                   '--config_path','/content/drive/MyDrive/chatbot/pretrain-model/output_BTS_re+pr05190/model/config.json',
                                   '--bert_model','/content/drive/MyDrive/chatbot/BertSum/pretrained_model/bert-base-chinese'])


    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device=torch.device("cpu")
    with open(args.config_path, 'r') as f:
        config = json.load(f)
    model = BertAbsSum(args.bert_model, config['decoder_config'], device)
    model.load_state_dict(torch.load(args.model_path))
    model.to(device)

    processor = LCSTSProcessor()
    tokenizer = BertTokenizer.from_pretrained(os.path.join(args.bert_model, 'vocab.txt'))
    # test_examples = processor.get_examples(args.eval_path)
    post=input("input:")
    test_examples = [InputExample(guid=0,src=convert_to_unicode(post),tgt=convert_to_unicode(post)),InputExample(guid=1,src=convert_to_unicode(post),tgt=convert_to_unicode(post))]
    test_features = convert_examples_to_features(test_examples, args.max_src_len, args.max_tgt_len, tokenizer)
    test_data = create_dataset(test_features)

    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE, drop_last=True)

    model.eval()
    # if not os.path.exists(args.result_path):
    #     os.mkdir(args.result_path)
    # f_log = open(os.path.join(args.result_path, 'log.txt'), 'w', encoding='utf-8')

    hyp_list = []
    ref_list = []
    for batch in tqdm(test_dataloader, desc="Iteration"):
        batch = tuple(t.to(device) for t in batch)
        pred, beam_score = model.beam_decode(batch[0], batch[1], 3, 3)
        src, tgt = batch[0], batch[2]
        for i in range(BATCH_SIZE):
            sample_src = "".join(tokenizer.convert_ids_to_tokens(src[i].cpu().numpy())).split('[CLS]')[1].split('[SEP]')[0] + '\n'
            sample_tgt = "".join(tokenizer.convert_ids_to_tokens(tgt[i].cpu().numpy())).split('[CLS]')[1].split('[SEP]')[0] + '\n'
            sample_pred = "".join(tokenizer.convert_ids_to_tokens(pred[i][0])).split('[SEP]')[0] + '\n'
            sample_pred_2 = "".join(tokenizer.convert_ids_to_tokens(pred[i][1])).split('[SEP]')[0] + '\n'
            sample_pred_3 = "".join(tokenizer.convert_ids_to_tokens(pred[i][2])).split('[SEP]')[0] + '\n'

            print('Hypothesis: ' + sample_pred)



            print('score: ' + str(beam_score[i]))
            
            # f_hyp.write(sample_pred)
            # f_ref.write(sample_tgt)


        


05/25/2021 15:35:04 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /content/drive/MyDrive/chatbot/BertSum/pretrained_model/bert-base-chinese
05/25/2021 15:35:04 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

05/25/2021 15:35:11 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file /content/drive/MyDrive/chatbot/BertSum/pretrained_model/bert-base-chinese/vocab.txt


input:hello


Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

lbk
[tensor([0.0775, 0.0794, 0.3002])]
tensor(0.0775)
lbk
[tensor([0.0775, 0.0794, 0.3002])]
tensor(0.0794)
lbk
[tensor([0.0775, 0.0794, 0.3002])]
tensor(0.3002)
lbk
[tensor([0.0775, 0.0794, 0.3002]), tensor([0.0662, 0.0929, 0.0834])]
tensor(0.0662)


IndexError: ignored